# import necessary packages

In [ ]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os.path import basename, join

# Define the path to the data directory

In [ ]:
path = "path/to/directory"

# Get all ".txt" files in the directory

In [ ]:
files = sorted(glob.glob(join(path, '*.txt')))

# Check if there are exactly 72 files (24 locations * 3 years)

In [ ]:
if len(files) != 72:
    print(f"Expected 72 files (24 locations * 3 years), found {len(files)} files.")
    exit()

# Organize files by location and year

In [ ]:
locations = sorted(set([basename(file).split('_')[0] for file in files]))

# Define a dictionary to map locations to zones

In [ ]:
location_zones = {
    'Zuarungu' : 'Sudan',
    'Navrongo' : 'Sudan',
    'Kintampo' : 'Transition',
    'Sunyani' : 'Transition',
    'Wenchi': 'Transition',
    'Bole' : 'Guinea', 
    'Kete' : 'Guinea',
    'Tamale' : 'Guinea',
    'Wa' : 'Guinea',
    'Yendi' : 'Guinea',
    'Accra': 'Coast',
    'Ada' : 'Coast',
    'Akatsi' : 'Coast',
    'Axim' : 'Coast',
    'Saltpond' : 'Coast',
    'Takoradi' : 'Coast',
    'Tema' : 'Coast',
    'Abetifi' : 'Forest',
    'Akim' : 'Forest',
    'Akuse' : 'Forest',
    'Ho' : 'Forest',
    'Koforidua' : 'Forest',
    'Kumasi' : 'Forest',
    'Sefwi' : 'Forest',
}

# Group locations by zone

In [ ]:
zones = {}
for location in locations:
    zone = location_zones.get(location, 'unknown')
    if zone not in zones:
        zones[zone] = []
    zones[zone].append(location)

# Define a dictionary to map years to specific colors

In [ ]:
year_colors = {
    '1960': 'blue',
    '1991': 'orange',
    '2030': 'cyan'
}

# This part iterates over all cliamte zones and makes a plot for the three decades on one canvas for each location. In addition, it identifies the onset and cessation of each map plot.

In [ ]:
for zone, zone_locations in zones.items():
    num_locations = len(zone_locations)
    cols = 2  # Number of columns for subplots
    rows = (num_locations + cols - 1) // cols  # Calculate number of rows needed
    
    # Create a figure for the zone with subplots
    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(15, 7 * rows))
    axes = axes.flatten()  # Flatten in case there is only one row

    # Iterate over each location in the zone
    for ax, location in zip(axes, zone_locations):
        location_files = [file for file in files if file.startswith(join(path, location))]
        
        # Check if there are exactly 3 files for each location
        if len(location_files) != 3:
            print(f"Skipping {location}: Expected 3 files, found {len(location_files)}")
            continue

        try:
            # Iterate over each file (year) for the location
            for file in location_files:
                # Read the data
                data = pd.read_csv(file, sep='\s+', header=None, names=['Julian Day', 'Cumm'])
                data.set_index('Julian Day', inplace=True)
                
                # Extract the year from the filename
                year = basename(file).split('_')[-2].split('.')[0]
                
                # Plot the data with the specified color and label
                data.plot(ax=ax, color=year_colors.get(year, 'cyan'), legend=True, label=year, linewidth=2.5)
                
                # Predict the onset and cessation
                anomaly = data['Cumm'].values.flatten()
                dx = 28
                onset = []
                cessation = []
                for i in range(dx, len(anomaly) - dx):
                    if min(anomaly[i-dx:i+dx]) == anomaly[i]:
                        onset.append([i+1, anomaly[i]])
                    if max(anomaly[i-dx:i+dx]) == anomaly[i]:
                        cessation.append([i+1, anomaly[i]])

                onset = np.array(onset).T
                cessation = np.array(cessation).T

                if onset.size > 0:
                    ax.plot(onset[0], onset[1], 'ko', label=f'Onset {year}', markersize=10)
                if cessation.size > 0:
                    ax.plot(cessation[0], cessation[1], 'ro', label=f'Cessation {year}', markersize=10)    
                    
            # Customize the plot for the location
            ax.set_title(location, fontsize=25)
            ax.grid(visible=True, which='both', axis='both', linestyle='-', linewidth=0.8)
            ax.minorticks_on()
            ax.set_ylabel('Cumulative daily rainfall anomaly', size=20)
            ax.set_xlabel('Julian day', size=20)
            ax.xaxis.set_tick_params(labelsize=20)
            ax.yaxis.set_tick_params(labelsize=20)

            
            # Create a custom legend
            custom_lines = [
                plt.Line2D([0], [0], color='blue', lw=5),
                plt.Line2D([0], [0], color='orange', lw=5),
                plt.Line2D([0], [0], color='cyan', lw=5)
            ]
            ax.legend(custom_lines, ['1960-1990', '1991-2020', '2030-2060'], fontsize=15)
        
        except Exception as e:
            print(f"Error processing {location}: {str(e)}")
        
    # Hide any unused subplots
    for i in range(num_locations, len(axes)):
        axes[i].set_visible(False)
    
    # Adjust layout and save the plot for the zone
    plt.tight_layout()
    plt.suptitle(f'Zone: {zone}', fontsize=20)
    plt.subplots_adjust(top=0.90, wspace=0.3, hspace=0.3)
    plt.savefig(join("path/to/directory",f'{zone}.png'))
    plt.show()
